In [1]:
from src.load import name_cleaner, financial_transfer_network_from_df, season_rankings_prem_league
from src.util import TransferNetwork, FinancialTransferNetwork

import pandas as pd
import networkx as nx

In [2]:
r = season_rankings_prem_league()
r.keys()

dict_keys([2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999])